# TEAM 73 (Pega a Visão)

INF0417 – 202108075 – ANDRÉ LUIS ARAÚJO DE SOUZA<br>
INF0417 – 202105848 – HUGO FERNANDES SILVA<br>
INF0417 – 202105865 – RAFAEL ALVES GOIAS<br>
INF0417 – 202105201 – VICTOR MATTEUS SILVA SOUZA<br>

## 1.
#### Formular 3 perguntas sobre o conteúdo da vídeo aula sobre extratores de características robustos com suas respectivas respostas. Deverá ter uma pergunta para cada um dos 3 níveis de aprendizado, sendo eles correspondentes aos tamanhos P (pequeno), M (médio) e G (grande). As perguntas e respostas serão avaliadas em função da clareza e coerência acorde com o nível requisitado para cada uma delas.

---
### **Pergunta de nível baixo**
#### Quais são os 3 passos que representam uma descrição do conceito de "local invariant feature"?

1º - Detecção: use um detector para achar um local (frame canônico)

2º - Description: Computa um descriptor pela amostragem da imagem no frame canônico.

3º - Matching: Acha correspondências, comparando descriptors (patches) entre duas imagens.

---
### **Pergunta de nível médio**
#### Qual é a limitação de busca força bruta pelo frame canônico numa imagem?

Complexidade combinatória pode gerar alto custo computacional principalmente quando se tem muitos pontos detectados em uma imagem e um número relativamente alto de pontos que satisfazem um frame, já que, por exemplo, se a homografia é 6DOF, é necessário detectar um frame canônico composto por N=3 pontos detectados os quais seus patches são correspondentes entre duas imagens, porém para encontrar esse frame canônico, considerando que foram detectados K pontos na imagem, o frame canônico de 3 pontos está em meio a análise combinatória de N em K.

---
### **Pergunta de alto nível**
#### Quais são os requisitos (em prol da invariância geométrica) que satisfazem a identificação de uma homografia 3DOF, 4DOF, 6DOF e 8DOF? I.e., o que compõe o frame canônico de cada um desses casos?

---
Mudança de escala + translação (3DOF)<br>
Em uma imagem, é necessária a identificação de 1 patch de pixels compostos pelos pixels da circunferência em que o diâmetro equivale a distância entre 2 pontos detectados numa imagem.<br>
É encontrada a homografia ocorrida de uma imagem para outra observando a transformação sofrida por esse patch que foi identificado como correspondente entre as duas imagens.<br>

---
Similaridade (mudança de escala + translação + rotação) (4DOF)<br>
É necessária a identificação de 2 patches de dois pontos detectados numa imagem.<br>
É encontrada a homografia de uma imagem para outra observando a transformação sofrida pelo triângulo retângulo em que as extremidades da hipotenusa são os 2 patches identificados<br>
Se só conseguir detectar 1 ponto, não conseguirá obter a homografia 4DOF.


---
Afim (mudança de escala + translação + rotação + torção/skew) (6DOF)<br>
É necessária a identificação de 3 patches de 3 pontos detectados numa imagem.<br>
É encontrada a homografia de uma imagem para outra observando a transformação sofrida pelo triângulo escaleno em que suas extremidades são os 3 patches identificados.<br>
Se só conseguir detectar 2 pontos, não conseguirá obter a homografia 6DOF.

---
Projetiva (mudança de escala + translação + rotação + torção + mudança de perspectiva/foreshortening) (8DOF)<br>
É necessária a identificação de 4 patches de 4 pontos detectados numa imagem.<br>
É encontrada a homografia de uma imagem para outra observando a transformação sofrida pelo plano em disposto em 3 dimensões em que os vértices são os 4 patches identificados.<br>
Se só conseguir detectar 3 pontos, não conseguirá obter a homografia 8DOF.

## 2.
#### Formular 1 exercício prático de programação de livre escolha sobre o tema de "rastreamento de objetos", ou sobre o tema de "feature extraction", junto com sua correspondente solução.

No contexto de rastreamento de objetos, o algoritmo MOG2 modela cada pixel na sequência de vídeo de entrada como uma mistura de distribuições gaussianas, onde o plano de fundo é representado por um conjunto de distribuições gaussianas. O algoritmo se adapta e atualiza esses modelos gaussianos ao longo do tempo para levar em conta as mudanças no plano de fundo devido a variações de iluminação, planos de fundo dinâmicos e outros fatores. Os objetos em primeiro plano são então detectados comparando a intensidade do pixel com os modelos gaussianos.

#### Implemente uma solução utilizando o algorítmo citado para detecção de objetos. (Utilize o arquivo 'video3.mp4' para testar a execução do código)

In [59]:
import cv2
import numpy as np

video_path = "video3.mp4"
cap = cv2.VideoCapture(video_path)

background_subtractor = cv2.createBackgroundSubtractorMOG2(history=15, detectShadows=False)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    mask = background_subtractor.apply(gray)
    
    _, thresh = cv2.threshold(mask, 128, 255, cv2.THRESH_TOZERO)

    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations=3)

    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 4000 and cv2.contourArea(contour) < 150000:  # Define uma área mínima para filtrar pequenos contornos
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Moving Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
